In [21]:
#necessary library
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import ModelSpec as MS, summarize, poly
from sklearn.model_selection import train_test_split

from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [28]:
#Load the data from ISLP package 
Portfolio = load_data('Portfolio')

#Make a function that inputs the dataframe D assumed to have cols x and y, as well as a vector
#idx indicating which observations should be used to estimate alpha 
#This function returns an estimate for alpha based on applying the min variance formula to the observations
#indexed by the argument idx.
def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0,1]) /
    (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))


In [30]:
print(Portfolio)

           X         Y
0  -0.895251 -0.234924
1  -1.562454 -0.885176
2  -0.417090  0.271888
3   1.044356 -0.734198
4  -0.315568  0.841983
..       ...       ...
95  0.479091  1.454774
96 -0.535020 -0.399175
97 -0.773129 -0.957175
98  0.403634  1.396038
99 -0.588496 -0.497285

[100 rows x 2 columns]


In [23]:
#alpha using all 100 observations.
alpha_func(Portfolio, range(100))

0.57583207459283

In [24]:
#randomly select 100 observations from range(100), with replacement.
rng = np.random.default_rng(0)
alpha_func(Portfolio,
rng.choice(100,100,replace=True))

0.6074452469619004

In [25]:
#Generalized to create boot_se function for bootstrap standard error for arbitrary function
#that takes only a data frame as an argument
#This is often used if the value of the counter is unimportant and simply makes sure the loop is executed B times.
def boot_SE(func,D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(len(D), len(D), replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)


In [26]:
# B = 1000 times 
alpha_SE = boot_SE(alpha_func, Portfolio, B=1000, seed=0)
alpha_SE

0.09118176521277699